# 图形问答

这个笔记将介绍如何对图形数据结构进行问答。

## 创建图形

在本节中，我们构建一个示例图形。目前，这对于小段文本效果最好。

In [1]:
from langchain.indexes import GraphIndexCreator
from langchain.llms import OpenAI
from langchain.document_loaders import TextLoader

In [2]:
index_creator = GraphIndexCreator(llm=OpenAI(temperature=0))

In [3]:
with open("../../state_of_the_union.txt") as f:
    all_text = f.read()

我们只使用了一个小片段，因为提取知识三元组有点费时。

In [4]:
text = "\n".join(all_text.split("\n\n")[105:108])

In [5]:
text

'It won’t look like much, but if you stop and look closely, you’ll see a “Field of dreams,” the ground on which America’s future will be built. \nThis is where Intel, the American company that helped build Silicon Valley, is going to build its $20 billion semiconductor “mega site”. \nUp to eight state-of-the-art factories in one place. 10,000 new good-paying jobs. '

In [6]:
graph = index_creator.from_text(text)

我们可以检查创建的图形。

In [7]:
graph.get_triples()

[('Intel', '$20 billion semiconductor "mega site"', 'is going to build'),
 ('Intel', 'state-of-the-art factories', 'is building'),
 ('Intel', '10,000 new good-paying jobs', 'is creating'),
 ('Intel', 'Silicon Valley', 'is helping build'),
 ('Field of dreams',
  "America's future will be built",
  'is the ground on which')]

## 查询图形
现在，我们可以使用图形问答链来问图形的问题。

In [8]:
from langchain.chains import GraphQAChain

In [9]:
chain = GraphQAChain.from_llm(OpenAI(temperature=0), graph=graph, verbose=True)

In [10]:
chain.run("what is Intel going to build?")



> Entering new GraphQAChain chain...
Entities Extracted:
 Intel
Full Context:
Intel is going to build $20 billion semiconductor "mega site"
Intel is building state-of-the-art factories
Intel is creating 10,000 new good-paying jobs
Intel is helping build Silicon Valley

> Finished chain.


' Intel is going to build a $20 billion semiconductor "mega site" with state-of-the-art factories, creating 10,000 new good-paying jobs and helping to build Silicon Valley.'

## 保存图形
我们还可以保存和加载图形。

In [7]:
graph.write_to_gml("graph.gml")

In [8]:
from langchain.indexes.graph import NetworkxEntityGraph

In [9]:
loaded_graph = NetworkxEntityGraph.from_gml("graph.gml")

In [10]:
loaded_graph.get_triples()

[('Intel', '$20 billion semiconductor "mega site"', 'is going to build'),
 ('Intel', 'state-of-the-art factories', 'is building'),
 ('Intel', '10,000 new good-paying jobs', 'is creating'),
 ('Intel', 'Silicon Valley', 'is helping build'),
 ('Field of dreams',
  "America's future will be built",
  'is the ground on which')]